In [1]:
import json
import polars as pl

from perpv2_market_api.market_pipe import SNXMarketData
from perpv2_market_api.data_structs import SNXMarketSummaryStruct

In [2]:
cd ../

/home/evan/Documents/perpv2_market_api


/home/evan/Documents/perpv2_market_api/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# get current market data for the latest block
snx_pipe = SNXMarketData()

# query market data
snx_market_data: list[SNXMarketSummaryStruct] = snx_pipe.preprocess_raw_market_summary_array()

In [4]:
# convert dataclass attributes to a dictionary
snx_market_list = [market.to_dict() for market in snx_market_data]

# load dataframe from dictionary
snx_markets_df = pl.from_dicts(snx_market_list)

In [5]:
# Specify the file path
file_path = 'data/perp_market_params.json'

# Read the JSON file
with open(file_path, 'r') as json_file:
    perp_market_params_data = json.load(json_file)

market_params_df = pl.from_dicts(perp_market_params_data).with_columns(pl.col("marketKey").alias("key")).select(['key', 'maxLeverage', 'maxMarketValue', 'maxFundingVelocity', 'skewScale'])

In [6]:
# join params to market data
snx_market_df = snx_markets_df.join(market_params_df, on="key", how='left')

In [7]:
complete_market_df = snx_pipe.transform_df(snx_market_df)

In [9]:
complete_market_df.schema

OrderedDict([('market', Utf8),
             ('asset', Utf8),
             ('key', Utf8),
             ('maxLeverage', Float64),
             ('price', Float64),
             ('marketSize', Float64),
             ('marketSkew', Float64),
             ('marketDebt', Float64),
             ('currentFundingRate', Float64),
             ('currentFundingVelocity', Float64),
             ('takerFeeOffchainDelayedOrder', Float64),
             ('makerFeeOffchainDelayedOrder', Float64),
             ('block', Int64),
             ('timestamp', Int64),
             ('eightHrFundingRate', Float64),
             ('eightHrFundingVelocity', Float64),
             ('yearlyFundingRate', Float64),
             ('long_oi', Float64),
             ('short_oi', Float64),
             ('relative_market_skew', Float64),
             ('marketSize_usd', Float64),
             ('marketSkew_usd', Float64),
             ('marketDebt_usd', Float64),
             ('long_oi_usd', Float64),
             ('short_oi_us

In [11]:
complete_market_df.shape

(74, 45)

In [10]:
complete_market_df.head(5)

market,asset,key,maxLeverage,price,marketSize,marketSkew,marketDebt,currentFundingRate,currentFundingVelocity,takerFeeOffchainDelayedOrder,makerFeeOffchainDelayedOrder,block,timestamp,eightHrFundingRate,eightHrFundingVelocity,yearlyFundingRate,long_oi,short_oi,relative_market_skew,marketSize_usd,marketSkew_usd,marketDebt_usd,long_oi_usd,short_oi_usd,maxLeverage_right,maxMarketValue,maxFundingVelocity,skewScale,premium_0,executionPrice,price_impact_full_rebalance,datetime,yearlyFundingVelocity,relative_market_skew_corrected_percent,total_marketSize_usd,total_marketSkew_usd,total_long_oi_usd,total_short_oi_usd,proportional_marketSize_usd,proportional_marketSkew_usd,proportional_long_oi_usd,proportional_short_oi_usd,maxMarketValue_usd,skewScale_maxMarketValue_multiplier
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""0x0940B0A96C5e…","""BNB""","""sBNBPERP""",27.5,231.327756,8817.896454,-193.099655,787957.059311,0.002246,-0.010904,0.0008,0.0002,112534747,1700668271,0.000749,-0.003635,81.997073,4312.3984,-4505.498054,-0.021899,2.0398e6,-44669.309766,1.8228e8,997577.444815,-1.0422e6,27.5,20000.0,36.0,637500.0,-0.000303,231.292721,-0.000151,2023-11-22 15:51:11,-398.012464,-0.021899,1.7044e8,345783.714011,8.5394e7,-8.5048e7,0.011968,-0.129183,0.011682,0.012255,20000.0,31.875
"""0x3a52b2181616…","""AXS""","""sAXSPERP""",27.5,5.734327,3919.299908,656.126746,6931.624943,-0.001377,0.006299,0.001,0.0002,112534747,1700668271,-0.000459,0.0021,-50.271881,2287.713327,-1631.586581,0.167409,22474.545596,3762.445035,39748.201087,13118.495315,-9356.05028,27.5,100000.0,36.0,3.75e6,0.000175,5.734828,0.000087,2023-11-22 15:51:11,229.906812,0.167409,1.7044e8,345783.714011,8.5394e7,-8.5048e7,0.000132,0.010881,0.000154,0.00011,100000.0,37.5
"""0x2B3bb4c683BF…","""sETH""","""sETHPERP""",55.0,2031.295,28355.345037,-186.332401,1.3217e7,0.001492,-0.001677,0.0006,0.0002,112534747,1700668271,0.000497,-0.000559,54.457399,14084.506318,-14270.838719,-0.006571,5.7598e7,-378496.074728,2.6847e10,2.8610e7,-2.8988e7,55.0,36000.0,9.0,1e6,-0.000186,2031.105752,-0.000093,2023-11-22 15:51:11,-61.210194,-0.006571,1.7044e8,345783.714011,8.5394e7,-8.5048e7,0.337934,-1.094604,0.335033,0.340846,36000.0,27.777778
"""0x59b007E9ea8F…","""sBTC""","""sBTCPERP""",55.0,36509.402,1098.032717,24.76808,9.2295e6,-0.000677,0.002229,0.0006,0.0002,112534747,1700668271,-0.000226,0.000743,-24.701888,561.400398,-536.632318,0.022557,4.0089e7,904267.791857,3.3697e11,2.0496e7,-1.9592e7,55.0,1750.0,9.0,100000.0,0.000248,36513.923339,0.000124,2023-11-22 15:51:11,81.363143,0.022557,1.7044e8,345783.714011,8.5394e7,-8.5048e7,0.235203,2.615125,0.240022,0.230365,1750.0,57.142857
"""0x31A1659Ca00F…","""LINK""","""sLINKPERP""",27.5,14.249132,1.0266e6,8023.082705,3.7201e6,0.000636,0.019255,0.0008,0.0002,112534747,1700668271,0.000212,0.006418,23.222301,517295.802628,-509272.719923,0.007815,1.4628e7,114321.968114,5.3008e7,7.3710e6,-7.2567e6,27.5,900000.0,36.0,1.5e7,0.000535,14.252943,0.000267,2023-11-22 15:51:11,702.822045,0.007815,1.7044e8,345783.714011,8.5394e7,-8.5048e7,0.085822,0.330617,0.086318,0.085325,900000.0,16.666667
